In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [3]:
def train_lightgbm_model(df, include_ratios=False, cv_folds=5, extrapolation_filenames=None):
    """
    Trains a LightGBM model on censored=0 data and validates on all data and a specific
    extrapolation set to assess generalizability. Uses relative error metrics.

    Parameters:
    - df (pandas.DataFrame): Preprocessed DataFrame with solver features.
    - include_ratios (bool): If True, includes ratio features; if False, excludes them (default: False).
    - cv_folds (int): Number of cross-validation folds (default: 5).
    - extrapolation_filenames (list): List of filenames for the extrapolation test set.

    Returns:
    - dict: Contains relative error metrics, CV scores, and feature importance for censored=0 test set,
            all data, and the new extrapolation set.
    """
    try:
        # Validate input
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Input 'df' must be a pandas DataFrame, got {type(df)}")
        print(f"Input DataFrame type: {type(df)}")

        # Define the extrapolation filenames if not provided
        if extrapolation_filenames is None:
            extrapolation_filenames = [
                'n27k3_v1.txt', 'n27k3_v2.txt', 'n27k3_v3.txt', 'n27k3_v4.txt',
                'n28k3_v1.txt', 'n28k3_v2.txt', 'n28k3_v3.txt', 'n28k3_v4.txt',
                'n28k3_v5.txt', 'n29k3_v1.txt', 'n29k3_v2.txt'
            ]
        
        # Separate extrapolation data
        df_extrapolation = df[df['filename'].isin(extrapolation_filenames)]
        df_model_data = df[~df['filename'].isin(extrapolation_filenames)]
        
        if df_extrapolation.empty:
            print("Warning: Extrapolation data not found in the DataFrame.")
            
        print(f"Total instances in original dataset: {len(df)}")
        print(f"Instances held for extrapolation: {len(df_extrapolation)}")
        print(f"Instances used for training/validation: {len(df_model_data)}")

        # Feature selection
        exclude_cols = ['filename', 'final_expandEvts', 'stop_iter', 'final_maxStackDepth']
        if not include_ratios:
            exclude_cols.extend([f'expandEvts_ratio_{i}' for i in range(1, 6)])
            exclude_cols.extend([f'pruneBacktrackEvts_ratio_{i}' for i in range(1, 6)])
        features = [col for col in df.columns if col not in exclude_cols and col != 'censored']
        
        # Print target statistics (on the model data, excluding extrapolation)
        print("\nTarget Statistics (final_expandEvts):")
        mean_censored = df_model_data[df_model_data['censored'] == 0]['final_expandEvts'].mean()
        std_censored = df_model_data[df_model_data['censored'] == 0]['final_expandEvts'].std()
        mean_all = df_model_data['final_expandEvts'].mean()
        std_all = df_model_data['final_expandEvts'].std()
        print(f"Mean (censored=0): {mean_censored:.2f}")
        print(f"Std Dev (censored=0): {std_censored:.2f}")
        print(f"Mean (all data): {mean_all:.2f}")
        print(f"Std Dev (all data): {std_all:.2f}")
        
        # Define relative error metrics
        def calculate_smape(y_true, y_pred):
            """Calculate Symmetric Mean Absolute Percentage Error (sMAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0
            if mask.sum() == 0:
                return np.nan
            smape = np.mean(2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask]) * 100
            return smape
        
        def calculate_smdape(y_true, y_pred):
            """Calculate Symmetric Median Absolute Percentage Error (sMdAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0
            if mask.sum() == 0:
                return np.nan
            smape_terms = 2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask] * 100
            return np.median(smape_terms)
        
        def calculate_normalized_rmse(y_true, y_pred, mean_y):
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            return rmse / mean_y if mean_y != 0 else np.nan
        
        # Train on censored=0 data (from the model data set)
        print("\nTraining LightGBM on censored=0 data...")
        df_censored = df_model_data[df_model_data['censored'] == 0]
        if df_censored.empty:
            print("Warning: No censored=0 instances found for training. Exiting.")
            return None
        
        X_censored = df_censored[features]
        y_censored = df_censored['final_expandEvts']
        X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_censored, y_censored, test_size=0.2, random_state=42)
        
        lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
        lgb_model.fit(X_train_c, y_train_c)
        
        # Evaluate on censored=0 test set
        y_pred_c = lgb_model.predict(X_test_c)
        rmse_censored = np.sqrt(mean_squared_error(y_test_c, y_pred_c))
        norm_rmse_censored = calculate_normalized_rmse(y_test_c, y_pred_c, mean_censored)
        smape_censored = calculate_smape(y_test_c, y_pred_c)
        smdape_censored = calculate_smdape(y_test_c, y_pred_c)
        cv_scores = cross_val_score(lgb_model, X_censored, y_censored, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_censored = (-cv_scores.mean()) / mean_censored if mean_censored != 0 else np.nan
        print(f"Censored=0 Test Set RMSE: {rmse_censored:.4f}")
        print(f"Censored=0 Test Set Normalized RMSE: {norm_rmse_censored:.4%}")
        print(f"Censored=0 Test Set sMAPE: {smape_censored:.2f}%")
        print(f"Censored=0 Test Set sMdAPE: {smdape_censored:.2f}%")
        print(f"Censored=0 CV Normalized RMSE: {cv_norm_rmse_censored:.4%}")
        
        # Feature importance
        importance = pd.DataFrame({
            'feature': X_censored.columns,
            'importance': lgb_model.feature_importances_
        }).sort_values('importance', ascending=False)
        print("\nCensored=0 Feature Importance (Top 10):")
        print(importance.head(10))
        
        # Validate on all data (from the model data set)
        print("\nValidating on all data (excluding extrapolation set)...")
        X_all = df_model_data[features]
        y_all = df_model_data['final_expandEvts']
        y_pred_all = lgb_model.predict(X_all)
        rmse_all = np.sqrt(mean_squared_error(y_all, y_pred_all))
        norm_rmse_all = calculate_normalized_rmse(y_all, y_pred_all, mean_all)
        smape_all = calculate_smape(y_all, y_pred_all)
        smdape_all = calculate_smdape(y_all, y_pred_all)
        print(f"All Model Data RMSE: {rmse_all:.4f}")
        print(f"All Model Data Normalized RMSE: {norm_rmse_all:.4%}")
        print(f"All Model Data sMAPE: {smape_all:.2f}%")
        print(f"All Model Data sMdAPE: {smdape_all:.2f}%")

        # Cross-validation for all model data
        print("\nPerforming cross-validation on all model data...")
        cv_scores_all = cross_val_score(lgb_model, X_all, y_all, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_all = (-cv_scores_all.mean()) / mean_all if mean_all != 0 else np.nan
        print(f"All Model Data CV Normalized RMSE: {cv_norm_rmse_all:.4%}")

        # Evaluate on the held-out extrapolation data
        print("\nEvaluating on held-out extrapolation data...")
        X_extrapolation = df_extrapolation[features]
        y_extrapolation = df_extrapolation['final_expandEvts']
        y_pred_extrapolation = lgb_model.predict(X_extrapolation)

        mean_extrapolation = y_extrapolation.mean()
        rmse_extrapolation = np.sqrt(mean_squared_error(y_extrapolation, y_pred_extrapolation))
        norm_rmse_extrapolation = calculate_normalized_rmse(y_extrapolation, y_pred_extrapolation, mean_extrapolation)
        smape_extrapolation = calculate_smape(y_extrapolation, y_pred_extrapolation)
        smdape_extrapolation = calculate_smdape(y_extrapolation, y_pred_extrapolation)
        
        print(f"Extrapolation Set RMSE: {rmse_extrapolation:.4f}")
        print(f"Extrapolation Set Normalized RMSE: {norm_rmse_extrapolation:.4%}")
        print(f"Extrapolation Set sMAPE: {smape_extrapolation:.2f}%")
        print(f"Extrapolation Set sMdAPE: {smdape_extrapolation:.2f}%")
        
        return {
            'rmse_censored_test': rmse_censored,
            'norm_rmse_censored_test': norm_rmse_censored,
            'smape_censored_test': smape_censored,
            'smdape_censored_test': smdape_censored,
            'cv_norm_rmse_censored': cv_norm_rmse_censored,
            'rmse_all': rmse_all,
            'norm_rmse_all': norm_rmse_all,
            'smape_all': smape_all,
            'smdape_all': smdape_all,
            'cv_norm_rmse_all': cv_norm_rmse_all,
            'feature_importance': importance,
            'rmse_extrapolation': rmse_extrapolation,
            'norm_rmse_extrapolation': norm_rmse_extrapolation,
            'smape_extrapolation': smape_extrapolation,
            'smdape_extrapolation': smdape_extrapolation,
        }
    
    except Exception as e:
        print(f"Error in train_lightgbm_model: {e}")
        return None

# --- Main execution block ---
df = pd.read_excel("structured_data.xlsx")
results = train_lightgbm_model(df, include_ratios=True)
results

Input DataFrame type: <class 'pandas.core.frame.DataFrame'>
Total instances in original dataset: 690
Instances held for extrapolation: 11
Instances used for training/validation: 679

Target Statistics (final_expandEvts):
Mean (censored=0): 53227800.02
Std Dev (censored=0): 174281183.67
Mean (all data): 750029157.52
Std Dev (all data): 635567837.57

Training LightGBM on censored=0 data...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2975
[LightGBM] [Info] Number of data points in the train set: 239, number of used features: 62
[LightGBM] [Info] Start training from score 44120478.410042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

{'rmse_censored_test': 149992189.49292123,
 'norm_rmse_censored_test': 2.8179295298124223,
 'smape_censored_test': 165.4726965435505,
 'smdape_censored_test': 199.99897528049362,
 'cv_norm_rmse_censored': 1.8985397143264189,
 'rmse_all': 621821816.9106477,
 'norm_rmse_all': 0.829063524629337,
 'smape_all': 121.51186361036588,
 'smdape_all': 98.79289131754123,
 'cv_norm_rmse_all': 0.10961355982588501,
 'feature_importance':                       feature  importance
 47          coef_of_variation         149
 43                    min_num          66
 46         range_to_avg_ratio          62
 39                  total_sum          51
 41                   skewness          50
 ..                        ...         ...
 5            strengthenEvts_1           0
 4             backtrackEvts_1           0
 3        pruneBacktrackEvts_1           0
 2                expandEvts_1           0
 69  diff_pruneBacktrackEvts_5           0
 
 [70 rows x 2 columns],
 'rmse_extrapolation': 651858370